# Plot supernovae in the $\Sigma_{\rm SFR}{-}\Sigma_{*}$ plane to compare each SN type with the resolved star-forming main sequence.

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os.path
from astropy.utils.console import ProgressBar
import seaborn as sns
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import colors
from scipy import stats
import matplotlib.gridspec as gridspec
import warnings
warnings.filterwarnings('ignore')

Read in final SN sample and the delivery index.

In [ ]:
sn_samp   = pd.read_csv('../sne_project/samples/FINAL_SAMPLE.csv')
del_index = pd.read_csv('../sne_project/delivery_index_full.csv')

Define functions to convert W3 and NUV to SFR.

In [ ]:
# W3
def w3_SFR_tracer(pixel_flux_w3):
    """ Input: flux measured at pixel (W3)
        Output: surfance density of SFR 
        in M_sol yr-1 kpc-2
    """
    SFR_w3 = (3.77e-03)*((10**(-42.86)) / (10**(-42.9)))*(pixel_flux_w3)
    return(SFR_w3)

# NUV
def nuv_SFR_tracer(pixel_flux_nuv):
    """ Input: flux measured at pixel (NUV)
        Output: surfance density of SFR 
        in M_sol yr-1 kpc-2
    """
    SFR_nuv = (1.05e-01)*((10**(-43.24)) / (10**(-43.17)))*(pixel_flux_nuv)
    return(SFR_nuv)

Define a function that will calculate an ellipse with major axis r25.

In [ ]:
def ellipse(delta_ra, delta_dec, pa, incl, r25_rad):
    part1 = ((-delta_ra*math.cos(np.radians(pa+90)) + delta_dec*math.sin(np.radians(pa+90)))**2)/(r25_rad**2)
    part2 = ((-delta_ra*math.sin(np.radians(pa+90)) - delta_dec*math.cos(np.radians(pa+90)))**2)/(r25_rad*math.cos(np.radians(incl)))**2
    return(part1, part2)

Define a function that will see if the SNe fall within r25 of their hosts.

In [ ]:
def in_gal(samp):
    in_galaxy = []

    #bar = ProgressBar(len(samp), ipython_widget=True)
    for index, row in samp.iterrows():

            #bar.update()

            # grab ra, dec, R25, inclination, and position angle; get ra, dec, and R25 in radians
            ra_gal  = np.radians(row['RA_GAL'])
            dec_gal = np.radians(row['DEC_GAL'])
            r25     = np.radians(row['R25'])
            incl    = row['INCL']
            pa      = row['PA']

            # grab RA and DEC of SN; get them in radians
            ra_sn  = np.radians(row['RA'])
            dec_sn = np.radians(row['DEC'])

            # take difference in RA and DEC of galaxy and RA and DEC of SNe
            delta_ra = ra_gal - ra_sn
            delta_dec = dec_gal - dec_sn

            part1, part2 = ellipse(delta_ra, delta_dec, pa, incl, r25)
            e = part1 + part2

            # compare distance with eqtn for inclined, oblique ellipse
            if e <= 1:
                in_galaxy += [1]
            else:
                in_galaxy += [0]
    return(in_galaxy)

In [ ]:
# run the above function and make an extra column in the sample DataFrame
in_galaxy = in_gal(sn_samp)
sn_samp['IN_R25'] = in_galaxy

Drop the SNe that are not within r25 of their host galaxies.

In [ ]:
sn_samp_r25 = sn_samp[sn_samp['IN_R25'] == 1.]
len(sn_samp_r25['PGC'].drop_duplicates(keep='first')) # how big is the galaxy sample?

Split the SN sample by type.

In [ ]:
# SNe Ia sample
ia = sn_samp_r25[sn_samp_r25['TYPE'] == 'Ia']

# SNe II sample
bitmask = (((sn_samp_r25.TYPE == 'II') |
            (sn_samp_r25.TYPE == 'II P') |
            (sn_samp_r25.TYPE == 'IIL')))
ii = sn_samp_r25[bitmask]

# SNe Ib/c sample
bitmask = (((sn_samp_r25.TYPE == 'Ib') |
            (sn_samp_r25.TYPE == 'Ic') |
            (sn_samp_r25.TYPE == 'Ib/c') |
            (sn_samp_r25.TYPE == 'Ibc') |
            (sn_samp_r25.TYPE == 'IIb')))
ibc = sn_samp_r25[bitmask]

Define a function that will add SFR(W3+NUV) columns to the SN sample, will convert W1 to sigma star (eq. 10 in z0mgs paper), will multiply the axes by cos(i), and will make a W1 3*RMS cut.

In [ ]:
def sample_prep(sample):
    """ Convert each SN's W1 to sigma_star and calculate SFR (W3+NUV).
        Return lists of sigma_star and sigma_sfr values for each SN.
    """
    sigma_star  = []
    sigma_sfr = []
    for index, row in sample.iterrows():
        
        # 3*RMS cut
        if (row['W1'] < 3*row['RMS_W1']):
            row['W1'] == 3*row['RMS_W1']
            continue

        # calculate SFR (W3+NUV)
        w3  = w3_SFR_tracer(row['W3'])
        nuv = nuv_SFR_tracer(row['NUV'])
        sfr = w3+nuv

        # multiply by cos(i) for both and convert W1 to sigma star
        info = del_index[del_index['pgc_name'] == row['PGC']] # delivery index information on the host galaxy
        sigma_star.append(330*(np.float(info['mtol'])/0.5)*row['W1']*np.cos(np.radians(row['INCL'])))
        sigma_sfr.append(sfr*np.cos(np.radians(row['INCL'])))

    return(sigma_star, sigma_sfr)

In [ ]:
# run the above function for SNe Ia, II, and Ib/c.
ia_star, ia_sfr   = sample_prep(ia)
ii_star, ii_sfr   = sample_prep(ii)
ibc_star, ibc_sfr = sample_prep(ibc)

Tweak sample_prep() so that it splits up a SN distribution by galaxy type.

In [ ]:
def split_sample_prep(sample):
    """ Convert each SN's W1 to sigma_star and calculate SFR (W3+NUV).
        Return lists of sigma_star and sigma_sfr values for each SN.
        This will split up the sample by host galaxy type.
    """
    sigma_star_ellip  = []
    sigma_sfr_ellip   = []
    sigma_star_spiral = []
    sigma_sfr_spiral  = []
    sigma_star_irr    = []
    sigma_sfr_irr     = []
    for index, row in sample.iterrows():
            
        # 3*RMS cut
        if (row['W1'] < 3*row['RMS_W1']):
            row['W1'] == 3*row['RMS_W1']
            continue

        # calculate SFR (W3+NUV)
        w3  = w3_SFR_tracer(row['W3'])
        nuv = nuv_SFR_tracer(row['NUV'])
        sfr = w3+nuv

        # multiply by cos(i) for both and convert W1 to sigma star
        info = del_index[del_index['pgc_name'] == row['PGC']] # delivery index information on the host galaxy
        
        if row['T'] == np.nan:
            continue
            
        if (row['T'] < 2):
            sigma_star_ellip.append(330*(np.float(info['mtol'])/0.5)*row['W1']*np.cos(np.radians(row['INCL'])))
            sigma_sfr_ellip.append(sfr*np.cos(np.radians(row['INCL'])))
            
        if (row['T'] >= 2) & (row['T'] <= 6):
            sigma_star_spiral.append(330*(np.float(info['mtol'])/0.5)*row['W1']*np.cos(np.radians(row['INCL'])))
            sigma_sfr_spiral.append(sfr*np.cos(np.radians(row['INCL'])))
            
        if (row['T'] > 6):
            sigma_star_irr.append(330*(np.float(info['mtol'])/0.5)*row['W1']*np.cos(np.radians(row['INCL'])))
            sigma_sfr_irr.append(sfr*np.cos(np.radians(row['INCL'])))

    return(sigma_star_ellip, sigma_sfr_ellip, sigma_star_spiral, sigma_sfr_spiral, sigma_star_irr, sigma_sfr_irr)

In [ ]:
ia_star_ellip, ia_sfr_ellip, ia_star_spiral, ia_sfr_spiral, ia_star_irr, ia_sfr_irr = split_sample_prep(ia)
ii_star_ellip, ii_sfr_ellip, ii_star_spiral, ii_sfr_spiral, ii_star_irr, ii_sfr_irr = split_sample_prep(ii)
ibc_star_ellip, ibc_sfr_ellip, ibc_star_spiral, ibc_sfr_spiral, ibc_star_irr, ibc_sfr_irr = split_sample_prep(ibc)

Add SFR (W3+NUV) to the control tables. **This is writing to file, so no need to run again.**

In [ ]:
already_done = []
all_w1 = []
all_sfr = []

bar = ProgressBar(len(sn_samp_r25), ipython_widget=True)
for index, row in sn_samp_r25.iterrows():
    pgc  = row['PGC']
    r25  = row['R25']
    incl = row['INCL']
    
    # skip if SN type is less than 3*RMS
    if (row['W1'] < 3*row['RMS_W1']):
        print(row['NAME'],'',row['PGC'])
        bar.update()
        continue
    
    # if host galaxy was already completed, skip
    if pgc in already_done:
        bar.update()
        continue
    else:
        already_done.append(pgc)
    
    # read in the host galaxy control table
    control_table = pd.read_csv('galaxy_control_tables/2kpc/%s_2kpc.csv' % (pgc))
    
    if len(control_table) == 0.:
        print(pgc)
        bar.update()
        continue
        
    # make a SFR column
    sfr = []
    for index, row in control_table.iterrows():    
        # convert to SFR (W3+NUV)
        w3  = w3_SFR_tracer(row['W3'])
        nuv = nuv_SFR_tracer(row['NUV'])
        sfr.append(w3+nuv)
        
    # add the column to the control table
    control_table['SFR(W3+NUV)'] = sfr
    control_table.to_csv('galaxy_control_tables/2kpc/%s_2kpc.csv' % (pgc), index=False)
    bar.update()

Now we want to collect all the pixels within r25 of the host galaxies and save the SFR and sigma_star values to a list. **This outputs to a file, so no need to run again.**

In [ ]:
already_done = []
all_star = []
all_sfr = []

bar = ProgressBar(len(sn_samp_r25), ipython_widget=True)
for index, row in sn_samp_r25.iterrows():
    pgc  = row['PGC']
    r25  = row['R25']
    incl = row['INCL']
    
    # most likely culprit for the weird long line
    if pgc == 'PGC46957':
        bar.update()
        continue
    
    # skip if SN type is less than 3*RMS
    if (row['W1'] < 3*row['RMS_W1']):
        print(row['NAME'],'',row['PGC'])
        bar.update()
        continue
    
    # if host galaxy was already completed, skip
    if pgc in already_done:
        bar.update()
        continue
    else:
        already_done.append(pgc)
    
    # read in the host galaxy control table
    control_table = pd.read_csv('galaxy_control_tables/2kpc/%s_2kpc.csv' % (pgc))
    
    if len(control_table) == 0.:
        print(pgc)
        bar.update()
        continue
    
    # get rgrid
    hdulist = pyfits.open('/data/kant/0/leroy.42/allsky/convolved/2kpc/%s_w1_2kpc.fits' % (pgc))
    w = WCS(hdulist[0].header)
    mapp = hdulist[0].data.copy()
    dimx, dimy = np.shape(mapp)
    mapp = mapp.flatten()
    
    # get pixels inside just R25
    hdulist1 = pyfits.open('rgrid/2kpc/%s_2kpc_rgrid.fits' % (pgc))
    
    rad_map    = hdulist1[0].data.copy() # copy data out into variable
    w          = WCS(hdulist1[0].header)
    rad_map    = rad_map.flatten()
    loc_inside = np.where(rad_map <= r25)[0]
    radii      = rad_map[loc_inside].byteswap().newbyteorder() # radii array; big endian to little endian (fixes error)
    hdulist1.close()
    
    # RA, DEC MAP; get the RA and Dec coords inside R25
    xx, yy = np.meshgrid(range(dimx), range(dimy))
    xx = xx.flatten()
    yy = yy.flatten()

    xx_inside = xx[loc_inside]
    yy_inside = yy[loc_inside]
    ra, dec   = w.wcs_pix2world(xx_inside, yy_inside, 0)
    
    # get the indices where the radii are within r25
    indices = []
    for i in range(len(ra)):
        indices.append(np.where((control_table['RA'] == ra[i]) & (control_table['DEC'] == dec[i]))[0])
        
    indices_list = [i for arr in indices for i in arr]
    w1_r25  = np.array(control_table['W1'][indices_list])
    sfr_r25 = np.array(control_table['SFR(W3+NUV)'][indices_list])
    
    # get the indices where W1 >= 3*RMS
    rms = np.where(w1_r25 >= 3*control_table['RMS_W1'][0])[0]
    rms_indices_list = [i for i in rms]
        
    # collect the pixels within r25 AND with W1 values that are >= 3*RMS
    # multiply by cos(i) for both and convert W1 to sigma star
    info = del_index[del_index['pgc_name'] == pgc] # get the delivery index info on this galaxy
    all_star.append(330*(np.float(info['mtol'])/0.5)*w1_r25[rms_indices_list]*np.cos(np.radians(row['INCL'])))
    all_sfr.append(sfr_r25[rms_indices_list]*np.cos(np.radians(row['INCL'])))
    
    hdulist.close()
    bar.update()

Write the above output to file. **No need to re-run**.

In [ ]:
# get the sigma star and sigma SFR values into lists
all_star_list  = [i for series in all_star for i in series]
all_sfr_list   = [i for series in all_sfr for i in series]

# get the number of pixels inside each galaxy
numpix_star = [len(series) for series in all_star for i in series]
numpix_sfr  = [len(series) for series in all_sfr  for i in series]

# make sure we only have positive points
is_positive    = np.where((np.array(all_star_list) > 0) & (np.array(all_sfr_list) > 0))[0]
all_star_list  = np.array(all_star_list)[is_positive]
all_sfr_list   = np.array(all_sfr_list)[is_positive]

numpix_star = np.array(numpix_star)[is_positive]
numpix_sfr  = np.array(numpix_sfr)[is_positive]

# output this to a file!
star_sfr = pd.DataFrame()
star_sfr['sigma_star']  = all_star_list
star_sfr['sigma_sfr']   = all_sfr_list
star_sfr['numpix_star'] = numpix_star
star_sfr['numpix_sfr']  = numpix_sfr
star_sfr.to_csv('results/star_sfr.csv',index=False)

### Plot!

Host galaxy pixels were written to file. The SN pixels were saved to memory by the sample_prep() function, but that takes a second to run.

#### First, plot SNe together.

In [ ]:
# read in the galaxy pixels
star_sfr = pd.read_csv('../sne_project/results/star_sfr.csv')

# font stuff for the axes
font_axes = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 20,
            }

fig = plt.figure(figsize=(10,8))

# plot the host galaxy pixels
plt.hist2d(np.log10(star_sfr['sigma_star']),np.log10(star_sfr['sigma_sfr']),bins=100,cmap='gray_r',
           weights=1./star_sfr['numpix_star'], norm=colors.LogNorm(),zorder=2)

# get the color bar for the galaxy pixels
cb = plt.colorbar()
cb.set_label('data density',fontdict=font_axes,rotation=270,labelpad=25)

# plot the SN types (from the sample_prep() function)
plt.scatter(np.log10(ia_star),  np.log10(ia_sfr),s=130,color='deepskyblue', alpha=0.7,edgecolors='lightgray',label='SNe Ia',zorder=2)
plt.scatter(np.log10(ii_star),  np.log10(ii_sfr),s=130,color='red',alpha=0.7,edgecolors='lightgray',label='SNe II',zorder=2)
plt.scatter(np.log10(ibc_star), np.log10(ibc_sfr),s=130,color='orange',alpha=0.7,edgecolors='lightgray',label='SNe Ib/c',zorder=2)

# axes limits
plt.ylim(-4.1,0)
plt.xlim(0,4.0)

# legend, gridlines, axes labels, inward ticks, save the figure
cb.ax.tick_params(labelsize=16)
plt.legend(fontsize=20,loc='upper left')
plt.grid(linestyle='--',alpha=0.5,zorder=0)
plt.xlabel('$\log_{10}\Sigma_{\star}\ [\mathrm{M}_{\odot}\ \mathrm{pc}^{-2}]$', fontdict=font_axes)
plt.ylabel('$\log_{10}\Sigma_{\mathrm{SFR}}\ [\mathrm{M}_{\odot}\ \mathrm{yr}^{-1}\ \mathrm{kpc}^{-2}]$', fontdict=font_axes)
plt.tick_params(direction='in',axis='both', which='major', length=5,labelsize=16)
plt.tight_layout()
plt.savefig('../sne_project/results/sfr_vs_star_full.pdf',dpi=200)
plt.savefig('../sne_project/results/sfr_vs_star_full.png',dpi=200)

#### Now plot the types individually.

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(24,6))

# plot the gray pixels on all subplots
ax0 = axes[0].hist2d(np.log10(star_sfr['sigma_star']),np.log10(star_sfr['sigma_sfr']),bins=100,cmap='gray_r',
           weights=1./star_sfr['numpix_star'], norm=colors.LogNorm(),zorder=2)
ax1 = axes[1].hist2d(np.log10(star_sfr['sigma_star']),np.log10(star_sfr['sigma_sfr']),bins=100,cmap='gray_r',
           weights=1./star_sfr['numpix_star'], norm=colors.LogNorm(),zorder=2)
ax2 = axes[2].hist2d(np.log10(star_sfr['sigma_star']),np.log10(star_sfr['sigma_sfr']),bins=100,cmap='gray_r',
           weights=1./star_sfr['numpix_star'], norm=colors.LogNorm(),zorder=2)

# plot the SN types
axes[0].scatter(np.log10(ia_star),  np.log10(ia_sfr),s=130,color='deepskyblue', alpha=0.7,edgecolors='lightgray',label='SNe Ia',zorder=2)
axes[1].scatter(np.log10(ii_star),  np.log10(ii_sfr),s=130,color='red', alpha=0.7,edgecolors='lightgray',label='SNe II',zorder=2)
axes[2].scatter(np.log10(ibc_star), np.log10(ibc_sfr),s=130,color='orange', alpha=0.7,edgecolors='lightgray',label='SNe Ib/c',zorder=2)

# legend, gridlines, axes labels, inward ticks on all subplots
for i in range(3):
    axes[i].legend(fontsize=20,loc='upper left')
    axes[i].grid(linestyle='--',alpha=0.5,zorder=0)
    axes[i].set_xlabel('$\log_{10}\Sigma_{\star}\ [\mathrm{M}_{\odot}\ \mathrm{pc}^{-2}]$ ', fontdict=font_axes, fontsize=20)
    axes[i].tick_params(direction='in',axis='both', which='major', labelsize=16)
    axes[i].set_xlim([0,4.0])
    axes[i].set_ylim([-4.1,0])
axes[0].set_ylabel('$\log_{10}\Sigma_{\mathrm{SFR}}\ [\mathrm{M}_{\odot}\ \mathrm{yr}^{-1}\ \mathrm{kpc}^{-2}]$', fontdict=font_axes, fontsize=20)
    
# set the colorbar
cb0 = fig.colorbar(ax0[3],ax=axes[0])
cb1 = fig.colorbar(ax1[3],ax=axes[1])
cb2 = fig.colorbar(ax2[3],ax=axes[2])
cb0.ax.tick_params(labelsize=16)
cb1.ax.tick_params(labelsize=16)
cb2.ax.tick_params(labelsize=16)
cb2.set_label('data density',fontdict=font_axes,fontsize=20,rotation=270,labelpad=25)

# save the figure
plt.tight_layout()
plt.savefig('../sne_project/results/sfr_vs_star.pdf',dpi=200)
plt.savefig('../sne_project/results/sfr_vs_star.png',dpi=200)

### Create a KDE plot of $\Sigma_{\rm SFR}$ and $\Sigma_{\rm SFR} / \Sigma_{*}$ values for SNe Ia, II, and Ib/c. Then perform a T-test to compare the median of the $\Sigma_{\rm SFR}$ and $\Sigma_{\rm SFR} / \Sigma_{*}$ values between all SN types.

Calculate the median (50th percentile), 16th, and 84th percentile of the distributions.

#### $\Sigma_{\rm SFR}$

In [ ]:
# ia
ia_sfr_dict = {
    'SFR': ia_sfr,
}

ia_sfr_df = pd.DataFrame(ia_sfr_dict)

ia_star_dict = {
    'STAR': ia_star,
}

ia_star_df = pd.DataFrame(ia_star_dict)
ia_df = ia_sfr_df.merge(ia_star_df, left_index=True, right_index=True)

# --------------------------------------------------------------------- 

# ii
ii_sfr_dict = {
    'SFR': ii_sfr,
}

ii_sfr_df = pd.DataFrame(ii_sfr_dict)

ii_star_dict = {
    'STAR': ii_star,
}

ii_star_df = pd.DataFrame(ii_star_dict)

ii_df = ii_sfr_df.merge(ii_star_df, left_index=True, right_index=True)

# --------------------------------------------------------------------- 

# ibc
ibc_sfr_dict = {
    'SFR': ibc_sfr,
}

ibc_sfr_df = pd.DataFrame(ibc_sfr_dict)

ibc_star_dict = {
    'STAR': ibc_star,
}

ibc_star_df = pd.DataFrame(ibc_star_dict)
ibc_df = ibc_sfr_df.merge(ibc_star_df, left_index=True, right_index=True)

In [ ]:
# calculate the median
ia_sfr_log = np.log10(ia_df['SFR'])
ia_sfr_median = np.median(np.array(ia_sfr_log)[~np.isnan(ia_sfr_log)])
print(ia_sfr_median)

ii_sfr_log = np.log10(ii_df['SFR'])
ii_sfr_median = np.median(np.array(ii_sfr_log)[~np.isnan(ii_sfr_log)])
print(ii_sfr_median)
ibc_sfr_log = np.log10(ibc_df['SFR'])
ibc_sfr_median = np.median(np.array(ibc_sfr_log)[~np.isnan(ibc_sfr_log)])
print(ibc_sfr_median)

In [ ]:
# calculate the 16th, 50th, and 84th percentiles
p_ibc = np.sort(ibc_sfr_log[~np.isnan(ibc_sfr_log)])
p16 = np.percentile(p_ibc, 16)
p50 = np.percentile(p_ibc, 50)
p84 = np.percentile(p_ibc, 84)
print(p16,p50,p84)

In [ ]:
# perform the T-test
print(stats.ttest_ind(ia_sfr_log, ii_sfr_log, equal_var=False, nan_policy='omit'))
print(stats.ttest_ind(ia_sfr_log, ibc_sfr_log, equal_var=False, nan_policy='omit'))
print(stats.ttest_ind(ii_sfr_log, ibc_sfr_log, equal_var=False, nan_policy='omit'))

Plot.

In [ ]:
font_axes = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 25,
            }
sns.set_style('whitegrid')

fig = plt.figure(figsize=(10,7))
sfr_data = [ia_sfr_log, ii_sfr_log, ibc_sfr_log]
colors = ['tab:blue','tab:red','tab:orange']
labels = ['SNe Ia','SNe II','SNe Ib/c']
zorders = [4,3,2]
alphas=[0.3,0.3,0.3]
bars = [0.89,.85,.81]

for i in range(len(sfr_data)):
    ax = sns.kdeplot(sfr_data[i], shade=True,color=colors[i], linewidth=5,alpha=alphas[i], label=labels[i],zorder=zorders[i])   

    # plot the percentiles
    perc = np.sort(sfr_data[i][~np.isnan(sfr_data[i])])
    ax.scatter([np.percentile(perc, 50)],[bars[i]], edgecolors='black',facecolors='white', marker='o', lw=3, s=100, zorder=11)
    ax.plot([np.percentile(perc, 16), np.percentile(perc, 84)], [bars[i], bars[i]], color=colors[i], lw=5, zorder=10)
    
    if i == 2:
        ax.set_xlabel('$\log_{10}\Sigma_{\mathrm{SFR}}\ [\mathrm{M}_{\odot}\ \mathrm{yr}^{-1}\ \mathrm{kpc}^{-2}]$',fontdict=font_axes)
        ax.set_ylabel('Data density',fontdict=font_axes)
        ax.tick_params(direction='in',axis='both', which='major', labelsize=20)

ax.legend(fontsize=25, loc='upper right')

plt.tight_layout()
plt.savefig('../sne_project/results/sfr_kde.pdf',dpi=200)


#### $\Sigma_{\rm SFR} / \Sigma_{*}$

In [ ]:
# calculate the median
ia_ratio  = np.log10(ia_df['SFR']/(ia_df['STAR']*(10**(6))))
print(np.median(np.array(ia_ratio)[~np.isnan(ia_ratio)]))

ii_ratio  = np.log10(ii_df['SFR']/(ii_df['STAR']*(10**(6))))
print(np.median(np.array(ii_ratio)[~np.isnan(ii_ratio)]))

ibc_ratio = np.log10(ibc_df['SFR']/(ibc_df['STAR']*(10**(6))))
print(np.median(np.array(ibc_ratio)[~np.isnan(ibc_ratio)]))

In [ ]:
# calculate the 16th, 50th, and 84th percentiles
p_ibc = np.sort(ibc_ratio[~np.isnan(ibc_ratio)])
p16 = np.percentile(p_ibc, 16)
p50 = np.percentile(p_ibc, 50)
p84 = np.percentile(p_ibc, 84)
print(p16,p50,p84)

In [ ]:
# perform the T-test
print(stats.ttest_ind(ia_ratio, ii_ratio,  equal_var=False, nan_policy='omit'))
print(stats.ttest_ind(ia_ratio, ibc_ratio, equal_var=False, nan_policy='omit'))
print(stats.ttest_ind(ii_ratio, ibc_ratio, equal_var=False, nan_policy='omit'))

In [ ]:
ia_ratio_ellip  = np.log10(ia_df_ellip['SFR']/(ia_df_ellip['STAR']*(10**(6))))
ia_ratio_spiral  = np.log10(ia_df_spiral['SFR']/(ia_df_spiral['STAR']*(10**(6))))
ia_ratio_irr  = np.log10(ia_df_irr['SFR']/(ia_df_irr['STAR']*(10**(6))))

Plot.

In [ ]:
font_axes = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 25,
            }

fig = plt.figure(figsize=(10,7))
sfr_data = [ia_ratio, ii_ratio, ibc_ratio]
colors = ['tab:blue','tab:red','tab:orange']
labels = ['SNe Ia','SNe II','SNe Ib/c']
zorders = [4,3,2]
alphas=[0.3,0.3,0.3]
bars = [1.4,1.35,1.3]

for i in range(len(sfr_data)):
    ax = sns.kdeplot(sfr_data[i], shade=True,color=colors[i], linewidth=5,alpha=alphas[i], label=labels[i],zorder=zorders[i])
    
    # plot the percentiles
    perc = np.sort(sfr_data[i][~np.isnan(sfr_data[i])])
    ax.scatter([np.percentile(perc, 50)],[bars[i]], edgecolors='black',facecolors='white', marker='o', lw=3, s=100, zorder=11)
    ax.plot([np.percentile(perc, 16), np.percentile(perc, 84)], [bars[i], bars[i]], color=colors[i], lw=5, zorder=10)
    
    if i == 2:
        ax.set_xlabel('$\log_{10}(\Sigma_{\mathrm{SFR}}$ / $\Sigma_{*}}$) [yr$^{-1}$]',fontdict=font_axes)
        ax.set_ylabel('Data density',fontdict=font_axes)
        ax.tick_params(direction='in',axis='both', which='major', labelsize=20)
        
ax.legend(fontsize=25, loc='upper left')

sns.set_style('whitegrid')
plt.tight_layout()
plt.savefig('../sne_project/results/sfr_star_kde.pdf',dpi=200)

#### Re-do the above plots but split the SNe based on galaxy type

SNe Ia

In [ ]:
# Ia in elliptical galaxies
ia_sfr_dict_ellip = {
    'SFR': ia_sfr_ellip,
}

ia_sfr_df_ellip = pd.DataFrame(ia_sfr_dict_ellip)

ia_star_dict_ellip = {
    'STAR': ia_star_ellip,
}

ia_star_df_ellip = pd.DataFrame(ia_star_dict_ellip)
ia_df_ellip = ia_sfr_df_ellip.merge(ia_star_df_ellip, left_index=True, right_index=True)

# --------------------------------------------------------------------- 

# Ia in spiral galaxies
ia_sfr_dict_spiral = {
    'SFR': ia_sfr_spiral,
}

ia_sfr_df_spiral = pd.DataFrame(ia_sfr_dict_spiral)

ia_star_dict_spiral = {
    'STAR': ia_star_spiral,
}

ia_star_df_spiral = pd.DataFrame(ia_star_dict_spiral)
ia_df_spiral = ia_sfr_df_spiral.merge(ia_star_df_spiral, left_index=True, right_index=True)

# --------------------------------------------------------------------- 

# Ia in irregular/dwarf galaxies
ia_sfr_dict_irr = {
    'SFR': ia_sfr_irr,
}

ia_sfr_df_irr = pd.DataFrame(ia_sfr_dict_irr)

ia_star_dict_irr = {
    'STAR': ia_star_irr,
}

ia_star_df_irr = pd.DataFrame(ia_star_dict_irr)
ia_df_irr = ia_sfr_df_irr.merge(ia_star_df_irr, left_index=True, right_index=True)

In [ ]:
ia_sfr_log_ellip = np.log10(ia_df_ellip['SFR'])
ia_star_log_ellip = np.log10(ia_df_ellip['STAR'])

ia_sfr_log_spiral = np.log10(ia_df_spiral['SFR'])
ia_star_log_spiral = np.log10(ia_df_spiral['STAR'])

ia_sfr_log_irr = np.log10(ia_df_irr['SFR'])
ia_star_log_irr = np.log10(ia_df_irr['STAR'])

SNe II

In [ ]:
# II in elliptical galaxies
ii_sfr_dict_ellip = {
    'SFR': ii_sfr_ellip,
}

ii_sfr_df_ellip = pd.DataFrame(ii_sfr_dict_ellip)

ii_star_dict_ellip = {
    'STAR': ii_star_ellip,
}

ii_star_df_ellip = pd.DataFrame(ii_star_dict_ellip)
ii_df_ellip = ii_sfr_df_ellip.merge(ii_star_df_ellip, left_index=True, right_index=True)

# --------------------------------------------------------------------- 

# II in spiral galaxies
ii_sfr_dict_spiral = {
    'SFR': ii_sfr_spiral,
}

ii_sfr_df_spiral = pd.DataFrame(ii_sfr_dict_spiral)

ii_star_dict_spiral = {
    'STAR': ii_star_spiral,
}

ii_star_df_spiral = pd.DataFrame(ii_star_dict_spiral)
ii_df_spiral = ii_sfr_df_spiral.merge(ii_star_df_spiral, left_index=True, right_index=True)

# --------------------------------------------------------------------- 

# II in irregular/dwarf galaxies
ii_sfr_dict_irr = {
    'SFR': ii_sfr_irr,
}

ii_sfr_df_irr = pd.DataFrame(ii_sfr_dict_irr)

ii_star_dict_irr = {
    'STAR': ii_star_irr,
}

ii_star_df_irr = pd.DataFrame(ii_star_dict_irr)
ii_df_irr = ii_sfr_df_irr.merge(ii_star_df_irr, left_index=True, right_index=True)

In [ ]:
ii_sfr_log_ellip = np.log10(ii_df_ellip['SFR'])
ii_star_log_ellip = np.log10(ii_df_ellip['STAR'])

ii_sfr_log_spiral = np.log10(ii_df_spiral['SFR'])
ii_star_log_spiral = np.log10(ii_df_spiral['STAR'])

ii_sfr_log_irr = np.log10(ii_df_irr['SFR'])
ii_star_log_irr = np.log10(ii_df_irr['STAR'])

SNe Ib/c

In [ ]:
# Ib/c in elliptical galaxies
ibc_sfr_dict_ellip = {
    'SFR': ibc_sfr_ellip,
}

ibc_sfr_df_ellip = pd.DataFrame(ibc_sfr_dict_ellip)

ibc_star_dict_ellip = {
    'STAR': ibc_star_ellip,
}

ibc_star_df_ellip = pd.DataFrame(ibc_star_dict_ellip)
ibc_df_ellip = ibc_sfr_df_ellip.merge(ibc_star_df_ellip, left_index=True, right_index=True)

# --------------------------------------------------------------------- 

# Ib/c in spiral galaxies
ibc_sfr_dict_spiral = {
    'SFR': ibc_sfr_spiral,
}

ibc_sfr_df_spiral = pd.DataFrame(ibc_sfr_dict_spiral)

ibc_star_dict_spiral = {
    'STAR': ibc_star_spiral,
}

ibc_star_df_spiral = pd.DataFrame(ibc_star_dict_spiral)
ibc_df_spiral = ibc_sfr_df_spiral.merge(ibc_star_df_spiral, left_index=True, right_index=True)

# --------------------------------------------------------------------- 

# Ib/c in irregular/dwarf galaxies
ibc_sfr_dict_irr = {
    'SFR': ibc_sfr_irr,
}

ibc_sfr_df_irr = pd.DataFrame(ibc_sfr_dict_irr)

ibc_star_dict_irr = {
    'STAR': ibc_star_irr,
}

ibc_star_df_irr = pd.DataFrame(ibc_star_dict_irr)
ibc_df_irr = ibc_sfr_df_irr.merge(ibc_star_df_irr, left_index=True, right_index=True)

In [ ]:
ibc_sfr_log_ellip = np.log10(ibc_df_ellip['SFR'])
ibc_star_log_ellip = np.log10(ibc_df_ellip['STAR'])

ibc_sfr_log_spiral = np.log10(ibc_df_spiral['SFR'])
ibc_star_log_spiral = np.log10(ibc_df_spiral['STAR'])

ibc_sfr_log_irr = np.log10(ibc_df_irr['SFR'])
ibc_star_log_irr = np.log10(ibc_df_irr['STAR'])

In [ ]:
ia_ratio_ellip  = np.log10(ia_df_ellip['SFR']/(ia_df_ellip['STAR']*(10**(6))))
ia_ratio_spiral  = np.log10(ia_df_spiral['SFR']/(ia_df_spiral['STAR']*(10**(6))))
ia_ratio_irr  = np.log10(ia_df_irr['SFR']/(ia_df_irr['STAR']*(10**(6))))

ii_ratio_ellip  = np.log10(ii_df_ellip['SFR']/(ii_df_ellip['STAR']*(10**(6))))
ii_ratio_spiral  = np.log10(ii_df_spiral['SFR']/(ii_df_spiral['STAR']*(10**(6))))
ii_ratio_irr  = np.log10(ii_df_irr['SFR']/(ii_df_irr['STAR']*(10**(6))))

ibc_ratio_ellip  = np.log10(ibc_df_ellip['SFR']/(ibc_df_ellip['STAR']*(10**(6))))
ibc_ratio_spiral  = np.log10(ibc_df_spiral['SFR']/(ibc_df_spiral['STAR']*(10**(6))))
ibc_ratio_irr  = np.log10(ibc_df_irr['SFR']/(ibc_df_irr['STAR']*(10**(6))))

#### Simplify the split SNe based on galaxy type

In [ ]:
cc_sfr_log_ellip = list(ii_sfr_log_ellip) + list(ibc_sfr_log_ellip)
cc_sfr_log_late = list(ii_sfr_log_spiral) + list(ibc_sfr_log_spiral) + list(ii_sfr_log_irr) + list(ibc_sfr_log_irr)
ia_sfr_log_late = list(ia_sfr_log_spiral) + list(ia_sfr_log_irr)

In [ ]:
# calculate the 16th, 50th, and 84th percentiles
p_ia = np.sort(np.array(ia_sfr_log_late)[~np.isnan(np.array(ia_sfr_log_late))])
p16 = np.percentile(p_ia, 16)
p50 = np.percentile(p_ia, 50)
p84 = np.percentile(p_ia, 84)
print(p16,p50,p84)

In [ ]:
# perform the T-test
print('T < 2')
print(stats.ttest_ind(ia_sfr_log_ellip, cc_sfr_log_ellip,  equal_var=False, nan_policy='omit'))
print(' ')
print('T > 2')
print(stats.ttest_ind(ia_sfr_log_late, cc_sfr_log_late,  equal_var=False, nan_policy='omit'))

In [ ]:
font_axes = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 25,
            }
sns.set_style('whitegrid')

fig,axes = plt.subplots(1,2,sharey=True,figsize=(15,7))
axes = axes.flatten()

sfr_data = [[ia_sfr_log_ellip, cc_sfr_log_ellip], [ia_sfr_log_late, cc_sfr_log_late]]

colors = [['tab:blue','tab:orange'],['tab:blue','tab:orange'],['tab:blue','tab:orange']]
labels = [['SNe Ia (%s)' % len(ia_sfr_log_ellip),'CC SNe (%s)' % len(cc_sfr_log_ellip)],
          ['SNe Ia (%s)' % len(ia_sfr_log_late),'CC SNe (%s)' % len(cc_sfr_log_late)]]

zorders = [[3,2],[3,2],[3,2]]
alphas  = [[0.3,0.3],[0.3,0.3],[0.3,0.3]]
bars    = [[.65,.61],[.95,.91],[1.15,1.11]]

for i in range(len(sfr_data)):
    for j in range(len(sfr_data[i])):
        sns.kdeplot(sfr_data[i][j], shade=True,color=colors[i][j], linewidth=5,alpha=alphas[i][j], 
                    label=labels[i][j],ax=axes[i],zorder=zorders[i][j])   

        # plot the percentiles
        perc = np.sort(np.array(sfr_data[i][j])[~np.isnan(np.array(sfr_data[i][j]))])
        axes[i].scatter([np.percentile(perc, 50)],[bars[i][j]], edgecolors='black',facecolors='white', marker='o', lw=3, s=100, zorder=11)
        axes[i].plot([np.percentile(perc, 16), np.percentile(perc, 84)], [bars[i][j], bars[i][j]], color=colors[i][j], lw=5, zorder=10)
    
        axes[i].set_xlabel('$\log_{10}\Sigma_{\mathrm{SFR}}\ [\mathrm{M}_{\odot}\ \mathrm{yr}^{-1}\ \mathrm{kpc}^{-2}]$',fontdict=font_axes,fontsize=22)
        axes[i].tick_params(direction='in',axis='both', which='major', labelsize=20)
        axes[i].set_xlim(-5,1)
    
    if i == 0:
        axes[i].set_ylabel('Data density',fontdict=font_axes)
        axes[i].text(-4.2, 0.9, '$T < 2$', horizontalalignment='center',verticalalignment='center',
                     fontdict=font_axes, fontsize=30, bbox=dict(boxstyle="round",ec='lightgray',fc='None'))
        axes[i].legend(fontsize=19, loc='upper right')
        
    if i == 1:
        axes[i].set_ylabel('Data density',fontdict=font_axes)
        axes[i].text(-4.2, 0.9, '$T > 2$', horizontalalignment='center',verticalalignment='center',
                     fontdict=font_axes, fontsize=30, bbox=dict(boxstyle="round",ec='lightgray',fc='None'))
        axes[i].legend(fontsize=19, loc='upper right')
    


plt.tight_layout()
plt.savefig('../sne_project/results/sfr_type_kde_v2.pdf',dpi=200)


In [ ]:
ia_ratio_ellip  = np.log10(ia_df_ellip['SFR']/(ia_df_ellip['STAR']*(10**(6))))
ia_ratio_late   = list(np.log10(ia_df_spiral['SFR']/(ia_df_spiral['STAR']*(10**(6))))) + list(np.log10(ia_df_irr['SFR']/(ia_df_irr['STAR']*(10**(6)))))

cc_ratio_ellip = list(np.log10(ii_df_ellip['SFR']/(ii_df_ellip['STAR']*(10**(6))))) + list(np.log10(ibc_df_ellip['SFR']/(ibc_df_ellip['STAR']*(10**(6)))))
cc_ratio_late  = list(np.log10(ii_df_spiral['SFR']/(ii_df_spiral['STAR']*(10**(6))))) + list(np.log10(ibc_df_irr['SFR']/(ibc_df_irr['STAR']*(10**(6)))))


In [ ]:
# calculate the 16th, 50th, and 84th percentiles
p_ia = np.sort(np.array(ia_ratio_late)[~np.isnan(np.array(ia_ratio_late))])
p16 = np.percentile(p_ia, 16)
p50 = np.percentile(p_ia, 50)
p84 = np.percentile(p_ia, 84)
print(p16,p50,p84)

In [ ]:
# perform the T-test
print('T < 2')
print(stats.ttest_ind(ia_ratio_ellip, cc_ratio_ellip,  equal_var=False, nan_policy='omit'))
print(' ')
print('T > 2')
print(stats.ttest_ind(ia_ratio_late, cc_ratio_late,  equal_var=False, nan_policy='omit'))

In [ ]:
font_axes = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 25,
            }
sns.set_style('whitegrid')

fig,axes = plt.subplots(1,2,sharey=True,figsize=(15,7))
axes = axes.flatten()

sfr_data = [[ia_ratio_ellip, cc_ratio_ellip], [ia_ratio_late, cc_ratio_late]]

colors = [['tab:blue','tab:orange'],['tab:blue','tab:orange'],['tab:blue','tab:orange']]
labels = [['SNe Ia (%s)' % len(ia_ratio_ellip),'CC SNe (%s)' % len(cc_ratio_ellip)],
          ['SNe Ia (%s)' % len(ia_ratio_late),'CC SNe (%s)' % len(cc_ratio_late)]]

zorders = [[3,2],[3,2],[3,2]]
alphas  = [[0.3,0.3],[0.3,0.3]]
bars    = [[.95,.91],[1.3,1.25]]

for i in range(len(sfr_data)):
    for j in range(len(sfr_data[i])):
        sns.kdeplot(sfr_data[i][j], shade=True,color=colors[i][j], linewidth=5,alpha=alphas[i][j], 
                    label=labels[i][j],ax=axes[i],zorder=zorders[i][j])   

        # plot the percentiles
        perc = np.sort(np.array(sfr_data[i][j])[~np.isnan(np.array(sfr_data[i][j]))])
        axes[i].scatter([np.percentile(perc, 50)],[bars[i][j]], edgecolors='black',facecolors='white', marker='o', lw=3, s=100, zorder=11)
        axes[i].plot([np.percentile(perc, 16), np.percentile(perc, 84)], [bars[i][j], bars[i][j]], color=colors[i][j], lw=5, zorder=10)
    
        axes[i].set_xlabel('$\log_{10}(\Sigma_{\mathrm{SFR}}$ / $\Sigma_{*}}$) [yr$^{-1}$]',fontdict=font_axes,fontsize=22)
        axes[i].tick_params(direction='in',axis='both', which='major', labelsize=20)
        axes[i].set_xlim(-13,-8)
    
    if i == 0:
        axes[i].set_ylabel('Data density',fontdict=font_axes)
        axes[i].text(-12.2, 1.25, '$T < 2$', horizontalalignment='center',verticalalignment='center',
                     fontdict=font_axes, fontsize=30, bbox=dict(boxstyle="round",ec='lightgray',fc='None'))
        
    if i == 1:
        axes[i].set_ylabel('Data density',fontdict=font_axes)
        axes[i].text(-12.2, 1.25, '$T > 2$', horizontalalignment='center',verticalalignment='center',
                     fontdict=font_axes, fontsize=30, bbox=dict(boxstyle="round",ec='lightgray',fc='None'))
    


plt.tight_layout()
plt.savefig('../sne_project/results/sfr_star_type_kde_v2.pdf',dpi=200)


## Extra analysis.

### Calculate the width of the bins (of the grayscale pixels).

In [ ]:
star_sfr = pd.read_csv('results/star_sfr.csv')
h, xedge, yedge, _ = plt.hist2d(np.log10(star_sfr['sigma_star']),np.log10(star_sfr['sigma_sfr']),bins=100,cmap='gray_r',
           weights=1./star_sfr['numpix_star'],zorder=2)

print(yedge[1]-yedge[0])

In [ ]:
sn_samp

### Look at SNe at low Sigma_SFR/Sigma_star values.

In [ ]:
sample = sn_samp[(sn_samp['TYPE'] != 'Ia')]

sigma_star  = []
sigma_sfr = []
sne = []
tipe = []
gal = []
for index, row in sample.iterrows():

    # 3*RMS cut
    if (row['W1'] < 3*row['RMS_W1']):
        row['W1'] == 3*row['RMS_W1']
        continue

    # calculate SFR (W3+NUV)
    w3  = w3_SFR_tracer(row['W3'])
    nuv = nuv_SFR_tracer(row['NUV'])
    sfr = w3+nuv

    # multiply by cos(i) for both and convert W1 to sigma star
    info = del_index[del_index['pgc_name'] == row['PGC']] # delivery index information on the host galaxy
    sigma_star.append(330*(np.float(info['mtol'])/0.5)*row['W1']*np.cos(np.radians(row['INCL'])))
    sigma_sfr.append(sfr*np.cos(np.radians(row['INCL'])))
    sne.append(row['NAME'])
    tipe.append(row['TYPE'])
    gal.append(row['T'])

In [ ]:
test = pd.DataFrame()

test['NAME'] = sne
test['TYPE'] = tipe
test['T'] = gal
test['SFR'] = sigma_sfr
test['STAR'] = sigma_star
test['SFR/STAR'] = np.log10(np.array(sigma_sfr)/(np.array(sigma_star)*(10**(6))))

test[test['SFR/STAR'] < -10.5]